## Note:

I solved this prior to the infrastructure change that replaced the submission of arrays with base64 encoded strings. As such, this code would no longer run correctly even if the API endpoints were still available.

In [ ]:
import json
import numpy as np
import random
import requests
import time
from PIL import Image

def load_labels(path="labels.json"):
    with open(path, "r", encoding="utf-8") as f:
        labels = json.loads(f.read())
    l2i, i2l = {}, {}
    for k, v in labels.items():
        i = int(k)
        l = v.split(", ")[0].lower()
        l2i[l] = i
        i2l[i] = l
    return l2i, i2l

class Model:
    def __init__(self, l2i, i2l, version=2, target_label="granny smith"):
        self.suffix = {
            1: "",
            2: "-jpg",
            3: "-pixel"
        }[version]
        self.l2i = l2i
        self.i2l = i2l
        self.target = self.l2i[target_label]

    def predict(self, image_data):
        array = image_data.array_pil.tolist()
        b = 0
        while True:
            try:
                res = requests.post(f"http://granny{self.suffix}.advml.com/score", json={"data": array})
                res_json = res.json()
                print(res_json)
                if "flag" in res_json:
                    print(res_json["flag"])
                    image = image_data.to_image()
                    image.save(f"granny2.jpg")
                    return
                preds = [0] * 1000
                for prob, label in res_json["output"]:
                    i = self.l2i[label.lower()]
                    preds[i] = prob
                return preds[self.target]
            except Exception as e:
                b += 1
                time.sleep(3 * b)

class ImageData:
    def __init__(self, array_keras):
        self.array = array_keras
        self.shape = array_keras.shape

    @property
    def array_keras(self):
        return self.array

    @property
    def array_pil(self):
        return ImageData.array_keras_to_pil(self.array)

    def to_image(self):
        return Image.fromarray(self.array_pil)

    def copy(self):
        return ImageData(np.copy(self.array))

    def crossover(self, other):
        mask = np.random.uniform(0, 1, self.array.shape[:2]) < 0.5
        self.array[mask] = other.array[mask]

    def perturb(self, magnitude, n_pixels):
        y = np.random.randint(0, self.shape[1], n_pixels)
        x = np.random.randint(0, self.shape[1], n_pixels)
        v = np.random.normal(0.0, magnitude, size=(n_pixels, 3))
        self.array[y, x] += v
        self.array = np.clip(self.array, -1.0, 1.0)

    @staticmethod
    def from_path(path="timber_wolf.jpg", dim=224):
        image = Image.open(path)
        if dim is not None:
            image = image.resize((dim, dim), Image.Resampling.NEAREST)
        array_pil = np.array(image)
        array_keras = ImageData.array_pil_to_keras(array_pil)
        return ImageData(array_keras)

    @staticmethod
    def array_pil_to_keras(array_pil):
        array_keras = np.copy(array_pil).astype(np.float32)
        array_keras = ((array_keras / 255.0) - 0.5) * 2.0
        return array_keras

    @staticmethod
    def array_keras_to_pil(array_keras):
        array_pil = ((np.copy(array_keras) / 2.0) + 0.5) * 255.0
        array_pil = array_pil.astype(np.uint8)
        return array_pil

class GeneticAttack:
    def __init__(self, model, image, n_gen=32, n_pop=32, k=8, rate_mut=0.8, pert_mag=0.4, pert_pixels=20000, mut_mag=0.1, mut_pixels=1000):
        self.model = model
        self.image = image
        self.n_gen = n_gen
        self.n_pop = n_pop
        self.k = k
        self.rate_mut = rate_mut
        self.pert_mag = pert_mag
        self.pert_pixels = pert_pixels
        self.mut_mag = mut_mag
        self.mut_pixels = mut_pixels

    def attack(self):
        pop = self.init_pop()
        for i in range(self.n_gen):
            scores = self.eval_fitness(pop)
            parents = self.select_next_gen(pop, scores)
            children = []
            while len(children) < self.n_pop:
                parent0, parent1 = random.sample(parents, k=2)
                child = parent0.copy()
                child.crossover(parent1)
                if random.uniform(0, 1) < self.rate_mut:
                    child.perturb(self.pert_mag / 3.0, self.pert_pixels // 10)
                children.append(child)
            pop = children
            print(f"[{str(i).zfill(3)}] {round(max(scores), 5)}")
            pop[scores.index(max(scores))].to_image().save(f"./out/{str(i).zfill(3)}.jpg")
        scores = self.eval_fitness(pop)
        return pop[scores.index(max(scores))]

    def init_pop(self):
        pop = []
        for _ in range(self.n_pop):
            image = self.image.copy()
            image.perturb(self.pert_mag, self.pert_pixels)
            pop.append(image)
        return pop

    def eval_fitness(self, pop):
        return [self.model.predict(pop[i]) for i in range(self.n_pop)]

    def select_next_gen(self, pop, scores):
        new_pop = []
        ixs = list(range(self.n_pop))
        while len(new_pop) < self.n_pop:
            sel_ixs = random.sample(ixs, k=self.k)
            sel_scores = [scores[ix] for ix in sel_ixs]
            new_pop.append(pop[scores.index(max(sel_scores))])
        return new_pop

l2i, i2l = load_labels()
model = Model(l2i, i2l, version=2)
image = ImageData.from_path("../data/granny/timber_wolf.jpg")
attack = GeneticAttack(model, image, n_gen=512)
attack.attack()